In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize

In [3]:
filter=pd.read_csv(r'C:\Users\WindowPC1\Desktop\Rahul_sir_exps\2\train.csv',sep=',', encoding='latin-1')

In [4]:
filter

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [5]:
filter.isnull().sum()

qid              0
question_text    0
target           0
dtype: int64

In [6]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , How did Quebec nationalists see their province as a nation in the 1960s?                                      1
What is the best method to teach the pronunciation of English words to kids?                                  1
How much mark should we get to pass in NEET exam?                                                             1
Iâm 13 and overweight. Aside from having a healthy diet, what exercises can I do at home to lose weight?    1
How do you get perfect teeth without using braces?                                                            1
                                                                                                             ..
Why are all psychotherapists so bad at their jobs?                                                            1
How many babies are born each day in India?                                                                   1
What are the milk substitutes for baking a cake?                                        

In [8]:
filter.isnull().sum()

qid              0
question_text    0
target           0
dtype: int64

In [9]:
filter.drop(['qid'],1,inplace=True)

C:\Users\WindowPC1\AppData\Local\Temp/ipykernel_1588/120453861.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  filter.drop(['qid'],1,inplace=True)


In [10]:
filter.dtypes

question_text    object
target            int64
dtype: object

In [11]:
import tensorflow as tf
import keras
# pip install --user keras==2.3.1

print(tf.__version__)

2.2.0


Using TensorFlow backend.


In [12]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [13]:
# !pip install wget

In [14]:
# !python -m wget  http://nlp.stanford.edu/data/glove.42B.300d.zip

In [15]:
# !unzip /content/glove.42B.300d.zip 

In [16]:
# !rm /content/glove.42B.300d.zip

In [17]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [18]:
len(embeding_index['provide'])

300

In [19]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [20]:
len(embeding_index)

1917494

In [21]:
X=filter['question_text']
Y=filter['target']

In [22]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [23]:
class_weight=class_weight.compute_class_weight(class_weight = 'balanced',classes = np.unique(y_train), y = y_train)
print(class_weight)

[0.53290299 8.09809347]


In [24]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5329029908749855, 1: 8.098093466635666}

In [25]:
x_train

980985                             What is stock obligation?
653956     Where can I rent a boat in Georgia at 18? I ha...
742561     Since Chinese people aren't exposed to any sor...
1203368    What is the scope of copy editor as a professi...
843276     How difficult is it to study in Austria if you...
                                 ...                        
1272002           How does Quora's distribution system work?
102721     How can girl judge that a man/ boy is decent p...
745147     Are stay at home dads, where the woman makes t...
1112000    Is the faculty of IIT Kanpur the best for comp...
85987      Do I need an international driving permit to d...
Name: question_text, Length: 1044897, dtype: object

In [26]:
from nltk import word_tokenize

In [27]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WindowPC1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\WindowPC1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [28]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [29]:
np.percentile(sent_lens,99)

44.0

In [30]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [31]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'get': 32,
 'when': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'not': 43,
 'at': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'was': 48,
 'did': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'u

In [32]:
vocab_size

196362

In [33]:
seq_train_matrix.shape

(1044897, 44)

In [34]:
seq_train

[[2, 3, 699, 10067],
 [61, 15, 8, 1568, 4, 4178, 6, 3971, 44, 819, 8, 24, 18, 4178, 445, 2101],
 [324,
  245,
  37,
  820,
  4179,
  5,
  50,
  1868,
  7,
  4190,
  7,
  55,
  1578,
  3892,
  5696,
  41,
  47,
  323,
  82,
  1697,
  4,
  6154,
  817,
  5528,
  7,
  839,
  272,
  3892,
  5696,
  324,
  47,
  41,
  24,
  114,
  22533,
  101,
  88192,
  22534],
 [2,
  3,
  1,
  895,
  7,
  2061,
  4962,
  36,
  4,
  2941,
  762,
  80,
  8,
  1617,
  5,
  25,
  134,
  134,
  333,
  6,
  197,
  221],
 [9,
  609,
  3,
  17,
  5,
  184,
  6,
  5189,
  20,
  14,
  100,
  758,
  3416,
  10,
  1159,
  27,
  242,
  1034,
  137,
  27,
  1493],
 [2, 3, 1, 1391, 2291, 44, 7868, 402, 59, 1491, 610, 643, 6, 3183, 1094],
 [6761,
  497,
  616,
  5,
  1280,
  7002,
  497,
  5,
  505,
  111,
  3742,
  7,
  157,
  113,
  5,
  25,
  21,
  54,
  2,
  3,
  1,
  34,
  79,
  7,
  206,
  63,
  94,
  24108,
  111,
  316],
 [5460, 8635],
 [2, 3, 1085, 571, 11978],
 [26, 12582, 1355, 1705, 1415],
 [2, 12, 1, 4371, 

In [35]:
x_train.shape

(1044897,)

In [36]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [37]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [38]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [39]:
 len(embedding_matrix)

196363

In [40]:
embedding_matrix.shape

(196363, 300)

In [41]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.073375  ,  0.091995  ,  0.30491999, ...,  0.059471  ,
        -0.44626999, -0.54118001],
       [ 0.063128  , -0.29771   ,  0.47742   , ..., -0.35565999,
         0.31481001, -0.14323001]])

In [42]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [43]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 44)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 44, 300)           58908900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [57]:
filepath='./weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [58]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [59]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [60]:
model.fit(seq_train_matrix,y_train,
              epochs=15,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Train on 1044897 samples, validate on 261225 samples
Epoch 1/15
1044897/1044897 [==============================] - 2336s 2ms/step - loss: 0.2588 - accuracy: 0.8923 - val_loss: 0.2226 - val_accuracy: 0.9034

Epoch 00001: val_loss improved from inf to 0.22264, saving model to ./weights-01-0.2226.h5
Epoch 2/15
1044897/1044897 [==============================] - 2323s 2ms/step - loss: 0.2342 - accuracy: 0.8995 - val_loss: 0.2290 - val_accuracy: 0.9114

Epoch 00002: val_loss did not improve from 0.22264
Epoch 3/15
1044897/1044897 [==============================] - 2353s 2ms/step - loss: 0.2137 - accuracy: 0.9047 - val_loss: 0.2096 - val_accuracy: 0.9158

Epoch 00003: val_loss improved from 0.22264 to 0.20962, saving model to ./weights-03-0.2096.h5
Epoch 4/15
1044897/1044897 [==============================] - 2360s 2ms/step - loss: 0.1919 - accuracy: 0.9090 - val_loss: 0.2220 - val_accuracy: 0.9034

Epoch 00004: val_loss did not improve from 0.20962
Epoch 5/15
1044897/1044897 [===============

In [61]:
best_model = load_model('./weights-03-0.2096.h5')

In [62]:
best_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 44)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 44, 300)           58908900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [63]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9642664176413905

In [64]:
best_model.save('./model.h5')